In [ ]:
import lzma
import pickle
import cProfile
import pstats

from sc2.game_info import GameInfo

import numpy as np
from scipy import ndimage, signal, spatial
from skimage.measure import block_reduce
import plotly.express as px
import plotly.graph_objects as go

In [10]:
import sys
sys.path.append("../bot")

from utils.dijkstra import Point, shortest_paths_opt, DijkstraOutput

In [11]:
with open("../resources/Site Delta AIE.xz", "rb") as f:
    game_info: GameInfo = pickle.load(f)

In [12]:
pathing = game_info.pathing_grid.data_numpy.T
pathing = block_reduce(pathing, 2, np.max)
cost = np.where(pathing == 1, 1.0, np.inf)
px.imshow(cost)

In [13]:
np.random.seed(0)
cost = np.random.rand(*pathing.shape)
cost = ndimage.gaussian_filter(cost, 2)
cost = (cost - np.min(cost)) / (np.max(cost) - np.min(cost))
cost = np.where(pathing == 1, cost, np.inf)
px.imshow(cost)

In [14]:
sources = [Point((p / 2).rounded) for p in game_info.start_locations]
#sources.extend(Point((2 * game_info.map_center - p).rounded) for p in game_info.start_locations)
for _ in range(100):
    sources.append(tuple(np.random.randint((1, 1), np.array(pathing.shape)-1)))
sources

[(74, 21),
 (29, 60),
 (93, 66),
 (3, 94),
 (48, 65),
 (72, 70),
 (28, 67),
 (65, 5),
 (76, 43),
 (32, 93),
 (85, 41),
 (6, 68),
 (24, 30),
 (94, 35),
 (88, 62),
 (39, 52),
 (56, 97),
 (72, 81),
 (26, 47),
 (50, 49),
 (15, 69),
 (63, 5),
 (28, 45),
 (69, 7),
 (43, 25),
 (57, 53),
 (48, 49),
 (32, 25),
 (85, 55),
 (5, 39),
 (59, 60),
 (30, 63),
 (4, 3),
 (2, 51),
 (15, 30),
 (62, 25),
 (24, 89),
 (41, 5),
 (15, 65),
 (28, 3),
 (43, 92),
 (80, 88),
 (20, 68),
 (25, 83),
 (39, 12),
 (1, 50),
 (33, 95),
 (6, 55),
 (1, 59),
 (28, 52),
 (63, 62),
 (95, 77),
 (60, 39),
 (47, 32),
 (51, 38),
 (40, 95),
 (1, 50),
 (1, 61),
 (27, 65),
 (48, 53),
 (37, 88),
 (58, 27),
 (56, 58),
 (83, 22),
 (70, 55),
 (85, 22),
 (85, 41),
 (15, 70),
 (19, 35),
 (25, 75),
 (15, 75),
 (16, 24),
 (66, 63),
 (32, 1),
 (45, 55),
 (68, 66),
 (41, 25),
 (96, 28),
 (39, 50),
 (84, 27),
 (13, 86),
 (58, 24),
 (42, 34),
 (29, 45),
 (87, 76),
 (4, 3),
 (52, 92),
 (88, 67),
 (66, 65),
 (55, 64),
 (26, 30),
 (98, 91),
 (56, 7

In [15]:
paths = shortest_paths_opt(cost, sources, diagonal=False)
target = Point((game_info.map_center / 2).rounded)
path = paths.get_path(target)

In [16]:
result = np.where(pathing == 1, 0.0, np.inf)
for x, y in path:
    result[x, y] = 1.0
px.imshow(result)

In [17]:
%%timeit
shortest_paths_opt(cost, sources)

8.23 ms ± 111 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
%%timeit
shortest_paths_opt(cost, sources, diagonal=True)

14.7 ms ± 189 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
with cProfile.Profile() as prof:
    shortest_paths_opt(cost, sources, diagonal=True)
    stats = pstats.Stats(prof)
stats.sort_stats(pstats.SortKey.CUMULATIVE).print_stats()

         72228 function calls in 0.033 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.018    0.018    0.033    0.033 c:\Users\volke\PycharmProjects\12PoolBot\notebooks\../bot\utils\dijkstra.py:56(shortest_paths_opt)
     4997    0.005    0.000    0.009    0.000 {built-in method _heapq.heappop}
    50709    0.005    0.000    0.005    0.000 c:\Users\volke\PycharmProjects\12PoolBot\notebooks\../bot\utils\dijkstra.py:19(__lt__)
     4896    0.001    0.000    0.002    0.000 {built-in method _heapq.heappush}
     3817    0.002    0.000    0.002    0.000 c:\Users\volke\PycharmProjects\12PoolBot\notebooks\../bot\utils\dijkstra.py:47(_neighbours_diagonal)
     3817    0.001    0.000    0.001    0.000 c:\Users\volke\PycharmProjects\12PoolBot\notebooks\../bot\utils\dijkstra.py:38(_neighbours)
     3918    0.000    0.000    0.000    0.000 {method 'append' of 'list' objects}
        6    0.000    0.000    0.000    0.00